# PI-II: Demo QRF

A simple example of fitting a quantile regression forest (QRF) using the [quantile-forest](https://zillow.github.io/quantile-forest/) package to estimate prediction intervals.

Starter script for the [Prediction interval competition II: House price](https://www.kaggle.com/competitions/prediction-interval-competition-ii-house-price/overview) competition.

In [1]:
!pip install -q quantile-forest 2>/dev/null  # package for quantile regression forests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 83.3 MB/s eta 0:00:00


In [2]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import GradientBoostingRegressor
from typing import Tuple
import torch
from sklearn.metrics import mean_squared_error

In [3]:
import lightgbm as lgb
params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'device': 'cpu',
    'gpu_platform_id': 0,
    'gpu_device_id': 0,
    'learning_rate': 0.01,
    # …
}

In [4]:
random_state = 0
np.random.seed(random_state)

# Competition variables.
base_path = "/kaggle/input/prediction-interval-competition-ii-house-price/"
alpha = 0.1  # the specified competition alpha (i.e., 90% coverage)

In [5]:
# 1. 读入 CSV
df = pd.read_csv(base_path + "dataset.csv")

# 2. 删除指定列（不修改原 df 返回新对象）
df_new = df.drop(columns='sale_nbr')

# 3. 保存为新的 CSV（覆盖旧文件也可以）
df_new.to_csv('dataset_no_sale_nbr.csv', index=False)


In [6]:
#定义损失函数
def winkler_score(y_true, lower, upper, alpha=0.1, return_coverage=False):
    """Compute the Winkler Interval Score for prediction intervals.

    Args:
        y_true (array-like): True observed values.
        lower (array-like): Lower bounds of prediction intervals.
        upper (array-like): Upper bounds of prediction intervals.
        alpha (float): Significance level (e.g., 0.1 for 90% intervals).
        return_coverage (bool): If True, also return empirical coverage.

    Returns:
        score (float): Mean Winkler Score.
        coverage (float, optional): Proportion of true values within intervals.
    """
    y_true = np.asarray(y_true)
    lower = np.asarray(lower)
    upper = np.asarray(upper)

    width = upper - lower
    penalty_lower = 2 / alpha * (lower - y_true)
    penalty_upper = 2 / alpha * (y_true - upper)

    score = width.copy()
    score += np.where(y_true < lower, penalty_lower, 0)
    score += np.where(y_true > upper, penalty_upper, 0)

    if return_coverage:
        inside = (y_true >= lower) & (y_true <= upper)
        coverage = np.mean(inside)
        return np.mean(score), coverage

    return np.mean(score)

In [7]:
df = pd.read_csv(base_path + "dataset.csv", index_col="id", parse_dates=["sale_date"])
df_test = pd.read_csv(base_path + "test.csv", index_col="id", parse_dates=["sale_date"])

## Data Preparation

Prepare the data, including separating features and target, encoding categoricals, imputation, and simple feature engineering.

In [8]:
# Split features and target.
X = df.drop("sale_price", axis=1)
y = df["sale_price"]

# Split train/val and test.
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=random_state
)
X_test = df_test.copy()

In [9]:
#对类别特征的编码
cat_cols = X_train.select_dtypes(include=["object"]).columns.tolist()
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X_train[cat_cols] = encoder.fit_transform(X_train[cat_cols])
X_val[cat_cols] = encoder.transform(X_val[cat_cols])
X_test[cat_cols] = encoder.transform(X_test[cat_cols])

In [10]:
# 对缺失值的插补填充
num_cols = X_train.select_dtypes(include="number").columns.tolist()

num_imputer = SimpleImputer(strategy="median")
cat_imputer = SimpleImputer(strategy="most_frequent")


def impute(df, cols, imputer, fit=False):
    """Helper function for imputation."""
    if fit:
        return pd.DataFrame(imputer.fit_transform(df[cols]), columns=cols, index=df.index)
    else:
        return pd.DataFrame(imputer.transform(df[cols]), columns=cols, index=df.index)


X_train[num_cols] = impute(X_train, num_cols, num_imputer, fit=True)
X_val[num_cols] = impute(X_val, num_cols, num_imputer)
X_test[num_cols] = impute(X_test, num_cols, num_imputer)

X_train[cat_cols] = impute(X_train, cat_cols, cat_imputer, fit=True)
X_val[cat_cols] = impute(X_val, cat_cols, cat_imputer)
X_test[cat_cols] = impute(X_test, cat_cols, cat_imputer)

In [11]:
#一个编码器将销售日期转换为每年中的周数，并应用于训练集、验证集和测试集
class SaleDateEncoder(BaseEstimator, TransformerMixin):
    """Encode sale date as a week of the year feature."""

    def __init__(self, date_column="sale_date"):
        self.date_column = date_column

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.assign(
            **{
                "sale_week": lambda x: x["sale_date"].dt.isocalendar().week,
            }
        ).drop(columns=["sale_date"])
        return X

    def fit_transform(self, X):
        return self.fit(self, X).transform(X)


# Sale date encoding.
saledate_encoder = SaleDateEncoder(date_column="sale_date")
X_train = saledate_encoder.fit_transform(X_train)
X_val = saledate_encoder.transform(X_val)
X_test = saledate_encoder.transform(X_test)

## Model Fitting

Fit a QRF model and use it to estimate a nominal marginal coverage of 90% (quantiles 0.05 and 0.95).

In [12]:
class GBDTIntervalRegressor(BaseEstimator, RegressorMixin):
    """
    基于 LightGBM 的分位数回归器，支持多个分位点训练并输出预测区间。
    默认使用 LightGBM，保留 sklearn GBDT 兼容接口。
    """

    def __init__(
        self,
        backend: str = "lightgbm",   # 默认改为 lightgbm
        n_estimators: int = 2000,
        learning_rate: float = 0.01,
        max_depth: int = 10,          # LightGBM 用 -1 表示无深度限制
        quantiles: Tuple[float, float, float] = (0.04, 0.5, 0.96),
        **kwargs
    ):
        self.backend = backend
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.quantiles = quantiles
        self.kwargs = kwargs
        self.models_ = {}

    def fit(self, X, y):
        for q in self.quantiles:
            if self.backend == "sklearn":
                # sklearn GBDT
                from sklearn.ensemble import GradientBoostingRegressor
                model = GradientBoostingRegressor(
                    loss="quantile",
                    alpha=q,
                    n_estimators=self.n_estimators,
                    learning_rate=self.learning_rate,
                    max_depth=self.max_depth,
                    **self.kwargs
                )
            else:
                # LightGBM 量化回归
                params = dict(
                    objective="quantile",
                    alpha=q,
                    n_estimators=self.n_estimators,
                    learning_rate=self.learning_rate,
                    max_depth=self.max_depth,
                    verbose=-1,
                    **self.kwargs
                )
                model = lgb.LGBMRegressor(**params)
            model.fit(X, y)
            self.models_[q] = model
        return self

    def predict(self, X) -> np.ndarray:
        lower = self.models_[self.quantiles[0]].predict(X)
        upper = self.models_[self.quantiles[2]].predict(X)
        # 返回一个 (n_samples, 2) 的数组，分别是下界和上界
        return np.stack((lower, upper), axis=1)

    @property
    def feature_importances_(self):
        # 返回所有分位数模型的重要性字典
        return {q: m.feature_importances_ for q, m in self.models_.items()}

    def get_params(self, deep=True):
        return {
            "backend": self.backend,
            "n_estimators": self.n_estimators,
            "learning_rate": self.learning_rate,
            "max_depth": self.max_depth,
            "quantiles": self.quantiles,
            **self.kwargs,
        }

    def set_params(self, **params):
        for k, v in params.items():
            setattr(self, k, v)
        return self


In [13]:
model = GBDTIntervalRegressor().fit(X_train, y_train)

In [14]:
y_val_pred = model.predict(X_val)
y_val_pred = pd.DataFrame(y_val_pred, columns=["pi_lower", "pi_upper"])
#产生每个样本的预测区间并保存

## Evaluation

Evaluate the QRF predictions on the validation data.

In [15]:
mws, coverage = winkler_score(
    y_val,
    y_val_pred["pi_lower"],
    y_val_pred["pi_upper"],
    alpha=alpha,
    return_coverage=True,
)

print("Mean Winkler Score:", round(mws, 2))
print("Coverage:", round(coverage * 100, 1), "%")

Mean Winkler Score: 827980.12
Coverage: 90.7 %


## Submission

In [16]:
# Predict intervals on test set.
test_preds = model.predict(X_test)

sample_submission = pd.read_csv(base_path + "sample_submission.csv")
sample_submission["pi_lower"] = test_preds[:, 0]
sample_submission["pi_upper"] = test_preds[:, 1]
sample_submission.to_csv("submission.csv",
                         index=False,
                         float_format="%.6f")

sample_submission

,id,pi_lower,pi_upper
0,200000,642011.810095,1.093347e+06
1,200001,614351.686402,1.713346e+06
2,200002,349692.663876,9.706599e+05
3,200003,218036.392805,5.469614e+05
4,200004,398831.808317,1.265153e+06
...,...,...,...
199995,399995,212100.007245,9.834077e+05
199996,399996,231725.122328,6.994209e+05
199997,399997,196062.032326,5.189138e+05
199998,399998,405223.707654,1.238750e+06
